In [56]:

# BlackJack - By Rickard Maus

import random
import numpy as np

deck = {"ace1" : 1, "ace2" : 1, "ace3" : 1, "ace4" : 1, "two1" : 2, "two2" : 2, "two3" : 2, "two4" : 2,
       "thr1" : 3, "thr2" : 3, "thr3" : 3, "thr4" : 3, "for1" : 4, "for2" : 4, "for3" : 4, "for4" : 4,
       "fiv1" : 5, "fiv2" : 5, "fiv3" : 5, "fiv4" : 5, "six1" : 6, "six2" : 6, "six3" : 6, "six4" : 6,
       "sev1" : 7, "sev2" : 7, "sev3" : 7, "sev4" : 7, "eit1" : 8, "eit2" : 8, "eit3" : 8, "eit4" : 8,
       "nin1" : 9, "nin2" : 9, "nin3" : 9, "nin4" : 9, "ten1" : 10, "ten2" : 10, "ten3" : 10, "ten4" : 10,
       "J1" : 10, "J2" : 10, "J3" : 10, "J4" : 10, "Q1" : 10, "Q2" : 10, "Q3" : 10, "Q4" : 10,
       "K1" : 10, "K2" : 10, "K3" : 10, "K4" : 10}

class b_jack():
    def __init__(self):
        self.deck_L = list(deck)
        self.Player_hand = []
        self.Dealer_hand = []
        self.hold = np.array([])
        self.Player_hand_size = 0
        self.Dealer_hand_size = 0
        self.reward = 0
        self.pull_c = 3
        self.n_Player = 2
        self.bustvar = False
        self.tie = False
        self.arr = np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]).reshape(1,13)
        
    def arr_start_assign(self):
        self.arr[0,0] = deck[self.Dealer_hand[1]]
        self.arr[0,1] = deck[self.Player_hand[0]]
        self.arr[0,2] = deck[self.Player_hand[1]]
        
    def player_hand_count(self):
        P_hand_s = len(self.Player_hand)
        self.Player_hand_size = 0
        for n in range(0, P_hand_s):
            self.Player_hand_size = self.Player_hand_size + deck[self.Player_hand[n]]
    
    def dealer_hand_count(self):
        D_hand_s = len(self.Dealer_hand)
        self.Dealer_hand_size = 0
        for n in range(0, D_hand_s):
            self.Dealer_hand_size = self.Dealer_hand_size + deck[self.Dealer_hand[n]]
        
    def bust(self, x):
        Bust = 0
        hand_s = len(x)
        for n in range(0, hand_s):
            Bust = Bust + deck[x[n]]
        if (Bust > 21) and (x == self.Player_hand):
            self.reward = -1
        elif (Bust > 21) and (x == self.Dealer_hand):
            self.bustvar = True
            self.reward = 1
    
    def deal(self, x):
        draw = ''.join(random.sample(self.deck_L,1))
        self.deck_L.remove(draw)
        if x%2 == 1:
            self.Player_hand.append(draw)
        else:
            self.Dealer_hand.append(draw)
            
    def game_start(self):
        for x in range(0,4):
            self.deal(x)
        self.arr_start_assign()

    def mid_game(self, x):
        if x == 0:
            self.reward = 0
            self.deal(1)
            self.bust(self.Player_hand)
            self.arr[0,self.pull_c] = deck[self.Player_hand[self.n_Player]]
            self.pull_c += 1
            self.n_Player += 1
        else:
            self.end_game()
        return self.arr, self.reward
            
    def end_game(self):
        for x in range(0,12):
            self.dealer_hand_count()
            self.player_hand_count()
            self.bust(self.Dealer_hand)
            if (self.bustvar == True):
                break
            elif (self.Dealer_hand_size < 12):
                self.deal(0)
            elif (self.Dealer_hand_size > self.Player_hand_size):
                self.reward = -1
                break
            elif (self.Dealer_hand_size == self.Player_hand_size):
                self.tie = True
                self.reward = 0
                break
            else:
                self.deal(0)
            

In [81]:
import tensorflow as tf
import time
learning_rate=0.000001

graph = tf.Graph()
with graph.as_default():
    tf_hand = tf.placeholder(tf.float32, shape=(1,13))
    nextQ = tf.placeholder(tf.float32, shape=[1,2])
    
    weights = {
        'W1' : tf.Variable(tf.random_uniform([13,13],0,1))
        ,'W2' : tf.Variable(tf.random_uniform([13,13],0,1))
        ,'W3' : tf.Variable(tf.random_uniform([13,2],0,1))
    }
    
    def Q_model(x):
        layer_1 = tf.matmul(x, weights['W1'])
        layer_2 = tf.matmul(tf.nn.relu(layer_1), weights['W2'])
        layer_out = tf.matmul(layer_2, weights['W3'])
        return layer_out
    

    loss = tf.reduce_sum(tf.square(nextQ - Q_model(tf_hand)))
    trainer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
    updateModel = trainer.minimize(loss)
    
    Q_values = Q_model(tf_hand)
    predict = tf.argmax(Q_values,1)
    
#inputs1 = tf.placeholder(shape=[1,13],dtype=tf.float32)
#W = tf.Variable(tf.random_uniform([13,2],0,0.01))
#Qout = tf.matmul(inputs1,W)
#predict = tf.argmax(Qout,1)

#nextQ = tf.placeholder(shape=[1,2],dtype=tf.float32)
#loss = tf.reduce_sum(tf.square(nextQ - Qout))
#trainer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
#updateModel = trainer.minimize(loss)


In [82]:
y = .9
e = 0.1
num_episodes = 10000000
jList = []
rList = []
actions = [0, 1]
tot_win = 0
win = 0
lose = 0

with tf.Session(graph=graph) as sess:
    tf.global_variables_initializer().run()
    for i in range(num_episodes):
        game = b_jack()
        game.game_start()
        s = game.arr
        rAll = 0
        d = False
        j = 0
        hey = False
        if (i % 50000 == 0):
            print(s)
        for j in range(1,13):
            prev = None
            game.hold,allQ = sess.run([predict,Q_values], feed_dict={tf_hand:s})
            if np.random.rand(1) < e:
                prev = game.hold
                game.hold[0] = random.sample(actions,1)[0]
            s1,r = game.mid_game(game.hold[0])
            if ((i % 50000) == 0):
                print(allQ)
                print(game.hold, prev)
                print(game.Dealer_hand, "Dealer hand")
                print(r, " r")
                print(s1)
                hey = True
                print("Winrate: {}%".format(100*win/50000))
                win = 0
            Q1 = sess.run(Q_values,feed_dict={tf_hand:s1})
            maxQ1 = np.max(Q1)
            targetQ = allQ
            targetQ[0,game.hold[0]] = r + y*maxQ1
            _,w1,w2,w3 = sess.run([updateModel,weights['W1'],weights['W2'],weights['W3']],feed_dict={tf_hand:s,nextQ:targetQ})
            s = s1
            if (r == 1) or game.tie:
                e = 1./((i/100000) + 10)
                if r == 1:
                    win += 1
                    tot_win += 1
                break
            elif (r == -1):
                lose += 1
                break
        if hey == True:
            print("---------------------")
        jList.append(j)
        rList.append(rAll)
print("Wins:", win)
print("Losses", lose)
print("Winrate after {} games lr={}: {}%".format(num_episodes, learning_rate, 100*win_tot/num_episodes))
            

[[ 7 10  8  0  0  0  0  0  0  0  0  0  0]]
[[ 460.96405029  496.96951294]]
[1] None
['J1', 'sev1', 'ace4'] Dealer hand
0  r
[[ 7 10  8  0  0  0  0  0  0  0  0  0  0]]
Winrate: 0.0%
---------------------
[[ 1 10 10  0  0  0  0  0  0  0  0  0  0]]
[[-0.0093317   0.10265541]]
[1] None
['thr2', 'ace3', 'for3', 'for4', 'for1', 'J1'] Dealer hand
1  r
[[ 1 10 10  0  0  0  0  0  0  0  0  0  0]]
Winrate: 14.268%
---------------------
[[8 6 7 0 0 0 0 0 0 0 0 0 0]]
[[-4.24556112 -4.20575333]]
[1] None
['nin1', 'eit2'] Dealer hand
-1  r
[[8 6 7 0 0 0 0 0 0 0 0 0 0]]
Winrate: 17.812%
---------------------
[[10 10  2  0  0  0  0  0  0  0  0  0  0]]
[[-3.61687946 -3.93281174]]
[0] None
['nin2', 'K4'] Dealer hand
0  r
[[10 10  2  7  0  0  0  0  0  0  0  0  0]]
Winrate: 20.002%
[[-3.73148823 -3.82910347]]
[0] None
['nin2', 'K4'] Dealer hand
-1  r
[[10 10  2  7  8  0  0  0  0  0  0  0  0]]
Winrate: 0.0%
---------------------
[[10  9  2  0  0  0  0  0  0  0  0  0  0]]
[[-3.36274028 -3.82309628]]
[0] None

KeyboardInterrupt: 

In [ ]:
------------------------------------------ #Pre-bugfix
Winrate after 100 games lr=0.001: 32.0%
Winrate after 1000 games lr=0.001: 34.4%
Winrate after 5000 games lr=0.001: 33.5%
Winrate after 10000 games lr=0.001: 33.3%
Winrate after 50000 games lr=0.001: 33.038%
Winrate after 100000 games lr=0.001: 33.431%
Winrate after 100000 games lr=0.0001: 37.839%
Winrate after 100000 games lr=0.0001: 37.83%
Winrate after 100000 games lr=0.00001: 39.87%
Winrate after 100000 games lr=0.000001: 39.173%
Winrate after 200000 games lr=0.000001: 39.3275%
Winrate after 500000 games lr=0.000001: 40.5666%
Winrate after 1000000 games lr=0.000001: 40.0185%
------------------------------------------ #Ties count as wins
Winrate after 100000 games lr=1e-06: 37.476%
Winrate after 200000 games lr=1e-06: 38.7175%
Winrate after 500000 games lr=1e-06: 39.7312%
Winrate after 500000 games lr=1e-05: 39.0798%
Winrate after 1000000 games lr=1e-06: 39.8523%
Winrate after 1000000 games lr=5e-06: 39.5439%
Winrate after 1000000 games lr=5e-07: 39.658%
Winrate after 1000000 games lr=8e-07: 39.9715%
------------------------------------------ 
Winrate after 1000000 games lr=7e-07: 26.6978%
Winrate after 1000000 games lr=7e-07: 25.1526%